# Import and Setup

## Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.preprocessing import StandardScaler

## Path and Data

In [2]:
path = r'C:\Users\rubix\Downloads\Machine Learning 1'

In [3]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'Dataset-weather-prediction-dataset-processed.csv'), index_col = False)

# Checks Before Scaling

In [4]:
# Define what a quick check does

def quick_data_check(df, n_top_values=5):
    print("=== Shape ===")
    print(df.shape)
    print("\n=== Data Types ===")
    print(df.dtypes)
    
    print("\n=== First 5 Rows ===")
    print(df.head())
    
    print("\n=== Missing Values ===")
    print(df.isnull().sum())
    
    print("\n=== Basic Stats (Numeric Columns) ===")
    print(df.describe())
    
    print("\n=== Extended Numeric Summary ===")
    print(df.select_dtypes(include='number').agg(['mean', 'median', 'min', 'max']))
    
    print("\n=== Constant Columns ===")
    constant_cols = [col for col in df.columns if df[col].nunique() == 1]
    print(constant_cols if constant_cols else "None")
    
    print("\n=== Mixed Data Types Check ===")
    for column in df.columns:
        inferred_type = pd.api.types.infer_dtype(df[column])
        print(f"Column '{column}': {inferred_type}")
        if inferred_type in ['mixed', 'mixed-integer']:
            print(f"  --> Column '{column}' has mixed data types.")  # Fixed indentation here
    
    print("\n=== Duplicate Row Counts ===")
    print(df.duplicated().value_counts())
    
    print("\n=== Top Values for Categorical Columns ===")
    for col in df.select_dtypes(include='object'):
        print(f"\nColumn: {col}")
        print(df[col].value_counts(dropna=False).head(n_top_values))
    
    print("\n=== Correlation Matrix (Numeric Only) ===")
    print(df.corr(numeric_only=True))

# Use quick_data_check(df) to call data

In [5]:
quick_data_check(df)

=== Shape ===
(22950, 170)

=== Data Types ===
DATE                     int64
MONTH                    int64
BASEL_cloud_cover        int64
BASEL_wind_speed       float64
BASEL_humidity         float64
                        ...   
VALENTIA_snow_depth      int64
VALENTIA_sunshine      float64
VALENTIA_temp_mean     float64
VALENTIA_temp_min      float64
VALENTIA_temp_max      float64
Length: 170, dtype: object

=== First 5 Rows ===
       DATE  MONTH  BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  \
0  19600101      1                  7               2.1            0.85   
1  19600102      1                  6               2.1            0.84   
2  19600103      1                  8               2.1            0.90   
3  19600104      1                  3               2.1            0.92   
4  19600105      1                  6               2.1            0.95   

   BASEL_pressure  BASEL_global_radiation  BASEL_precipitation  \
0           1.018                    0.32     

## Summary of quick_data_check(df)

Nothing too crazy at first glance. I could maybe make a case for the date to be NOT an int64 type, but I think it's fine at the moment with the preferred YYYYMMDD format. 

Obviously I can't see literally all 170 columns at once using this method, but at least I get the names. It might be worth checking to see if all locations have the same number of columns. 

The correlation matrix is also kind of funny. I'm sure the correlations are going to be silly, but maybe a heatmap to see it better? Might be better after scaling so let's do that first.

# Scaling

## Separate Data

I don't seen any object/string types so I shouldn't have to limit to numerical only, but we'll check anyway. Should be 22950, 170 when done.

In [6]:
df_num = df.select_dtypes(include=np.number)

In [7]:
df_num.shape

(22950, 170)

Passed, awesome. Since I don't need that, I'll go ahead and delete it.

In [8]:
del df_num

That being said, we may have to return to the standard format for the DATE and MONTH columns eventually for practical displaying of data. I'll skip it for now.

## Scaling

In [9]:
scaler = StandardScaler()
# Use the column names from the original dataframe
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

## Checks

In [10]:
df_scaled.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,-1.707663,-1.599964,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,...,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237
1,-1.707657,-1.599964,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,...,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141
2,-1.707652,-1.599964,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,...,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078
3,-1.707646,-1.599964,-1.001953,-0.02793,1.458455,-0.001949,-0.821838,-0.445514,-0.179228,-0.114186,...,0.783085,0.480538,0.387574,-1.183432,0.669056,-0.024706,-1.039536,-0.998679,-0.164486,-0.838511
4,-1.707641,-1.599964,0.244897,-0.02793,1.729466,-0.001949,-0.746661,-0.164944,-0.179228,0.187388,...,-1.670486,-0.363113,1.729970,-0.794645,-0.490810,-0.024706,0.672505,-1.509396,-1.339569,-1.471186


In [11]:
quick_data_check(df_scaled)

=== Shape ===
(22950, 170)

=== Data Types ===
DATE                   float64
MONTH                  float64
BASEL_cloud_cover      float64
BASEL_wind_speed       float64
BASEL_humidity         float64
                        ...   
VALENTIA_snow_depth    float64
VALENTIA_sunshine      float64
VALENTIA_temp_mean     float64
VALENTIA_temp_min      float64
VALENTIA_temp_max      float64
Length: 170, dtype: object

=== First 5 Rows ===
       DATE     MONTH  BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  \
0 -1.707663 -1.599964           0.660514          -0.02793        0.826097   
1 -1.707657 -1.599964           0.244897          -0.02793        0.735760   
2 -1.707652 -1.599964           1.076130          -0.02793        1.277781   
3 -1.707646 -1.599964          -1.001953          -0.02793        1.458455   
4 -1.707641 -1.599964           0.244897          -0.02793        1.729466   

   BASEL_pressure  BASEL_global_radiation  BASEL_precipitation  \
0       -0.001949           

## Summary of Scaling and Checks

Everything seems to be in order so far. No lost data and all of it seems to have a standard deviation of 1.

# Changes after assignment approval

My suspicion to keep the DATE and MONTH columns away from scaling was correct. In order to bring those back into the final table I'll have to make some modifications.

My first thought was concatenation, but, if I understand it correctly, data would be doubled when reintegrated and we don't want that. So I'm thinking merging is the next best bet.

However, I'll need to drop the DATE and MONTH columns from the scaled data first so they can be replaced. I'll need to choose a common column to merge on between the two DataFrames. The "third" DataFrame only needs to be 3 columns (DATE, MONTH, and the shared column). Once that prep is done, I can merge them and export the frame as a new file for future assignments.

My reasoning for this is not wanting to write the 168 columns to redo the scaling on the original dataframe.

Sidenote: Bummer that .csv files don't save datetime attributes. It makes sense, but still sucks.

Anyway, let's fix this stuff!

## Drop DATE and MONTH from df_scaled

In [12]:
df_scaled_fixed = df_scaled.drop(columns = ['DATE', 'MONTH'])

### Check

In [13]:
df_scaled_fixed.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,-0.528623,-0.845652,...,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237
1,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,-0.582946,-0.462450,...,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141
2,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,-0.257010,-0.186545,...,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078
3,-1.001953,-0.02793,1.458455,-0.001949,-0.821838,-0.445514,-0.179228,-0.114186,-0.555784,-0.385810,...,0.783085,0.480538,0.387574,-1.183432,0.669056,-0.024706,-1.039536,-0.998679,-0.164486,-0.838511
4,0.244897,-0.02793,1.729466,-0.001949,-0.746661,-0.164944,-0.179228,0.187388,-1.003946,-1.075573,...,-1.670486,-0.363113,1.729970,-0.794645,-0.490810,-0.024706,0.672505,-1.509396,-1.339569,-1.471186


Good to go!

## Choose a shared column to merge on

I'll go with BASEL_cloud_cover simply because it's the 3rd column, haha. I'll write an empty line for copying purposes.

In [14]:
# BASEL_cloud_cover

## New DataFrame based on the original with only DATE, MONTH, and BASEL_cloud_cover included

In [15]:
df_three_column = df[['DATE','MONTH','BASEL_cloud_cover']]

### Check

In [16]:
df_three_column.head()

,DATE,MONTH,BASEL_cloud_cover
0,19600101,1,7
1,19600102,1,6
2,19600103,1,8
3,19600104,1,3
4,19600105,1,6


In [17]:
df_three_column.shape

(22950, 3)

Good here too!

## Merge DataFrames

In [18]:
df_scaled_merged = df_three_column.merge(df_scaled_fixed, on='BASEL_cloud_cover', how='outer')

C:\Users\rubix\AppData\Local\Temp\ipykernel_19248\1791522954.py:1: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation.
  df_scaled_merged = df_three_column.merge(df_scaled_fixed, on='BASEL_cloud_cover', how='outer')


In [19]:
df_scaled_merged

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,NaN,NaN,-2.248803,-0.02793,-2.245358,-0.001949,-0.220423,-0.445514,-0.179228,0.976121,...,-2.283879,-1.909805,2.522357,-0.229137,-0.490810,-0.024706,1.363329,-1.749733,-1.940775,-1.298638
1,NaN,NaN,-2.248803,-0.02793,-1.161315,-0.001949,-0.220423,-0.445514,-0.179228,0.906527,...,-1.057093,-0.503721,1.963025,-0.523673,-0.490810,-0.024706,0.462254,-2.380618,-2.268705,-2.132619
2,NaN,NaN,-2.248803,-0.02793,0.013064,-0.001949,-0.198944,-0.445514,-0.179228,0.952923,...,0.783085,-1.066155,0.629951,-0.665050,-0.419798,-0.024706,-0.949429,-1.839859,-1.804137,-1.643734
3,NaN,NaN,-2.248803,-0.02793,-0.348283,-0.001949,0.853532,-0.445514,-0.179228,1.347289,...,-0.443701,-4.300148,-0.069214,0.630907,-0.490810,-0.024706,0.342111,0.593556,0.327410,0.800693
4,NaN,NaN,-2.248803,-0.02793,-1.161315,-0.001949,1.261635,-0.445514,-0.179228,1.625665,...,0.783085,0.199321,-1.309066,-0.394077,1.177978,-0.024706,-1.039536,0.082839,0.518703,-0.378383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45895,20221011.0,10.0,8.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45896,20221013.0,10.0,8.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45897,20221014.0,10.0,8.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45898,20221020.0,10.0,8.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Notes

Okay, normally I would cut out all of the above because it's not useful to the final result, but I'll leave it in for teaching purposes.

The BASEL_cloud_cover is actually NOT a shared column because it is mixing scaled and not scaled data. Back to the drawing board.

----------

Okay, after some thinking/research I'm going to try something new.

## Trying Something Different

In [20]:
# This should create a DataFrame called df_first_two that only contains values from the first two columns
df_first_two = df.iloc[:, :2]

# Let's check

In [21]:
df_first_two 

,DATE,MONTH
0,19600101,1
1,19600102,1
2,19600103,1
3,19600104,1
4,19600105,1
...,...,...
22945,20221027,10
22946,20221028,10
22947,20221029,10
22948,20221030,10


Great! Easy enough so far.

In [22]:
# This code creates a new NumPy array (not particularly obvious compared to a df) called df_rest_scaled.
# However, it also REMOVES the first two columns in the scaling process.
# This is slightly counter-intuitive because while it is scaling data on only the selected columns the columns it doesn't scale are removed
# I would've thought that a selective modification would not change the rest of the data and that's why the next steps take place.

df_rest_scaled = scaler.fit_transform(df.iloc[:, 2:])

In [23]:
# This code is transforming the array back to a DataFrame while preserving the column names and row alignment via index.

df_rest_scaled = pd.DataFrame(df_rest_scaled, 
                              columns=df.columns[2:], 
                              index=df.index)

In [24]:
# This code merges the two DataFrames via concat

df_scaled_fixed = pd.concat([df_first_two, df_rest_scaled], axis=1)

# We can see the result here:

df_scaled_fixed

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,...,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237
1,19600102,1,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,...,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141
2,19600103,1,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,...,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078
3,19600104,1,-1.001953,-0.02793,1.458455,-0.001949,-0.821838,-0.445514,-0.179228,-0.114186,...,0.783085,0.480538,0.387574,-1.183432,0.669056,-0.024706,-1.039536,-0.998679,-0.164486,-0.838511
4,19600105,1,0.244897,-0.02793,1.729466,-0.001949,-0.746661,-0.164944,-0.179228,0.187388,...,-1.670486,-0.363113,1.729970,-0.794645,-0.490810,-0.024706,0.672505,-1.509396,-1.339569,-1.471186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,20221027,10,-1.833187,-0.02793,0.284075,1.037367,-0.005632,-0.004619,-0.179228,0.720943,...,-0.443701,-0.081896,-0.003958,-0.005290,-0.005560,-0.024706,-0.018319,-0.007287,-0.000520,-0.004530
22946,20221028,10,0.244897,-0.02793,0.103401,0.976231,-0.005632,-0.004619,-0.179228,0.187388,...,-0.443701,-0.081896,-0.003958,-0.005290,-0.005560,-0.024706,-0.018319,-0.007287,-0.000520,-0.004530
22947,20221029,10,-0.586336,-0.02793,0.013064,0.716402,-0.005632,-0.004619,-0.179228,0.349774,...,-0.443701,-0.081896,-0.003958,-0.005290,-0.005560,-0.024706,-0.018319,-0.007287,-0.000520,-0.004530
22948,20221030,10,-0.170720,-0.02793,0.374412,0.487141,-0.005632,-0.004619,-0.179228,0.280180,...,-0.443701,-0.081896,-0.003958,-0.005290,-0.005560,-0.024706,-0.018319,-0.007287,-0.000520,-0.004530


Now we have scaled data while keeping DATE and MONTH "untouched." Time to export for future assignments.

# Export

In [25]:
df_scaled_fixed.to_csv(os.path.join(path, '02 Data','Prepared Data', 'weather_scaled.csv'), index=False)